In [65]:
import xml.etree.ElementTree as ET
import os
import pandas as pd
import numpy as np
import time
import csv
import json
from urllib2 import urlopen
import urllib2
import pdb
from itertools import product
from chemspipy import ChemSpider

In [63]:
# Make the table form of HMDB files. 'path' is the location which HMDB xml files are in. This HMDB xml files  
# (path는 HMDB에서 제공하는 zip 파일의 압축을 푼 폴더를 말함. 함수의 입력으로 받으나 여기서는 하드코딩함.)
path = "/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/4_HMDB/hmdb_metabolites/"
def parse_hmdb_xml(path):
    print "parse the HMDB xml files"
    print "Start :" + time.strftime('%a %H:%M:%S')
    # 폴더 내의 모든 파일 이름 얻어오기. 
    file_name = []
    for root, dirs, files in os.walk(path):
        for f in files:
            file_name.append(f)
        
    # HMDB00001 부터 시작하게 sort한다.
    file_name.sort()

    # 마지막은 전체 파일이기 때문에 삭제 한다. # hmdb_metabolites.xml을 제거하기 위해서이다.
    del file_name[len(file_name)-1]

    #file별 path
    file_path=[]
    for i in file_name:
        file_path.append(path+i)

    # xml 파일별로 필요한 정보 파싱
    df = pd.DataFrame()
    for j in file_path:
        #parse xml file
        tree = ET.parse(j)
        #get root node
        xml_root = tree.getroot()
        
        #필요한 데이터 저장
        #accession
        accession = xml_root[3].text
        #name
        if type(xml_root[5].text) == unicode:
            name = xml_root[5].text.encode("ascii", "replace")
        else:
            name = xml_root[5].text
        #pubchem cid
        pubchem_id = xml_root[43].text
        #smiles
        smiles = xml_root[14].text
        #inchi
        inchi = xml_root[15].text
        #inchikey
        inchikey = xml_root[16].text
        
        temp_df = pd.DataFrame([[accession, name, pubchem_id, smiles, inchi, inchikey]],columns = ['Accession', 'Name', 'PubChem_CID', 'Smiles', 'InchI', 'InchI_key'])
        df = df.append(temp_df)
    print "End :" + time.strftime('%a %H:%M:%S')
    df.index = range(0, len(df))
    #df.to_csv("/home/share")
    return df

In [66]:
#parse the HMDB xml files
temp = parse_hmdb_xml(path)
#df_hmdb = preprocess_raw_hmdb(df)

parse the HMDB xml files


In [ ]:
def get_chemspider_smiles(df):
    cs = ChemSpider('3e6c1209-d6af-49a8-8f8c-f9daec87a410')
    
    df = df.fillna('null')
    df.replace('InChIKey=','', regex=True, inplace = True)
    
    print "Start :" + time.strftime('%a %H:%M:%S')
    result = pd.DataFrame()
    search = pd.DataFrame()
    for index, row in df.iterrows():
        accession = row['Accession']
        name = row['Name']
        inchikey = row['InChI_key']
        inchi = row['InChI']
        smiles = row['Smiles']
    
        com = cs.simple_search(inchikey)    
        if com == []: # inchi key mapping결과 없을 때
            temp_df = pd.DataFrame([[accession, name, smiles, inchi]], columns = ['Accession', 'Name', 'Smiles', 'InChI'])
            search = search.append(temp_df)
        
        else:
            chem_smiles = com[0].smiles.encode('ascii', 'replace')
            temp_df = pd.DataFrame([[accession, name, chem_smiles]], columns = ['Accession', 'Name', 'ChemSpider_Smiles'])
            result = result.append(temp_df)
    print "InChI key end :" + time.strftime('%a %H:%M:%S')

    search2 = pd.DataFrame()
    for index, row in search.iterrows():
        accession = row['Accession']
        name = row['Name']
        inchi = row['InChI']
        smiles = row['Smiles']
    
        com = cs.simple_search(inchi)
    
        if com == []: # inchi key mapping결과 없을 때
            temp_df = pd.DataFrame([[accession, name, inchi]], columns = ['Accession', 'Name', 'Smiles'])
            search2 = search.append(temp_df)
        
        else:
            chem_smiles = com[0].smiles.encode('ascii', 'replace')
            temp_df = pd.DataFrame([[accession, name, chem_smiles]], columns = ['Accession', 'Name', 'ChemSpider_Smiles'])
            result = result.append(temp_df)
    print "InChI end :" + time.strftime('%a %H:%M:%S')

    for index, row in search2.iterrows():
        accession = row['Accession']
        name = row['Name']
        smiles = row['Smiles']
    
        com = cs.simple_search(inchi)
    
        if com == []: # inchi key mapping결과 없을 때
            continue
    
        else:
            chem_smiles = com[0].smiles.encode('ascii', 'replace')
            temp_df = pd.DataFrame([[accession, name, chem_smiles]], columns = ['Accession', 'Name', 'ChemSpider_Smiles'])
            result = result.append(temp_df)        
        
    result.index = range(len(result))

    print "End :" + time.strftime('%a %H:%M:%S')
    
    return result

In [64]:
def preprocess_raw_hmdb(df):
    print "preprocess the HMDB files"
    
    #cid 존재하는 hmdb
    df_hmdb_cid = df[~df.PubChem_CID.isnull()]

    # using UniChem REST API, convert pubchem ID to KEGG ID
    df_hmdb = pd.DataFrame()
    url_base = 'https://www.ebi.ac.uk/unichem/rest/src_compound_id/'
    cnt = 0
    print "pubchem ids of hmdb are converted to KEGG ID start : " + time.strftime('%a %H:%M:%S')
    for index, row in df_hmdb_cid.iterrows():
        accession = row['Accession']
        name = row['Name']
        hmdb_pubchem_id = row['PubChem_CID']
        
        url = url_base + hmdb_pubchem_id + '/22/6'
                   
        MAX_ATTEMPTS = 8
        for attempt in range(MAX_ATTEMPTS):
            try:
                f = urlopen(url)
            except urllib2.HTTPError, e:
                if e.code == 404:
                    cnt = cnt + 1
                    break
                else:
                    print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                    raise
            except urllib2.URLError, e:
                if e.args[0][0] == 110: # connection time out
                    sleep_secs = attempt ** 2
                    print e
                    print url
                    print '  cnt = %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    print '    Retrying in %d seconds...' % sleep_secs
                    time.sleep(sleep_secs)
                    continue
                else:
                    print e
                    print '   cnt : %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    raise
            else: # error가 일어나지 않을 때.
                raw_data = f.readlines()
                f.close()
                cnt = cnt + 1
                if raw_data[0] == '[]':
                    break
                else:
                    splited = raw_data[0].rstrip(']').lstrip('[').split(',')
                    for i in splited:
                        kegg_id = i.split(':')[1].lstrip("\"").rstrip("\"}")
                        if kegg_id != 'null':
                            temp_df = pd.DataFrame([[accession, name, hmdb_pubchem_id, kegg_id]], columns = ['Accession', 'Name', 'PubChem_CID', 'KEGG_ID'])
                            df_hmdb = df_hmdb.append(temp_df)
            
                break
        else: #connection을 8번 시도했는데 안될 경우.
            print 'we failed to reconnect 8 times in %s' % url 
    print "end : " + time.strftime('%a %H:%M:%S')
    
    df_hmdb.index = range(len(df_hmdb))
    return df_hmdb          

In [55]:
# Make the final form of HMDB files. 'path' is the location which HMDB xml files are in. This HMDB xml files  
# (path는 HMDB에서 제공하는 zip 파일의 압축을 푼 폴더를 말함. 함수의 입력으로 받으나 여기서는 하드코딩함.)
path = "/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/4_HMDB/hmdb_metabolites/"
print "preprocess the HMDB files"
  
# 폴더 내의 모든 파일 이름 얻어오기. 
file_name = []
for root, dirs, files in os.walk(path):
    for f in files:
        file_name.append(f)
        
# HMDB00001 부터 시작하게 sort한다.
file_name.sort()

# 마지막은 전체 파일이기 때문에 삭제 한다. # hmdb_metabolites.xml을 제거하기 위해서이다.
#del file_name[len(file_name)-1]

#file별 path
file_path=[]
for i in file_name:
    file_path.append(path+i)

# xml 파일별로 필요한 정보 파싱
df = pd.DataFrame()
for j in file_path:
    #parse xml file
    tree = ET.parse(j)
    #get root node
    xml_root = tree.getroot()
        
    #필요한 데이터 저장
    #accession
    accession = xml_root[3].text
    #name
    if type(xml_root[5].text) == unicode:
        name = xml_root[5].text.encode("ascii", "replace")
    else:
        name = xml_root[5].text
    #pubchem cid
    pubchem_id = xml_root[43].text
        
    temp_df = pd.DataFrame([[accession, name, pubchem_id]],columns = ['Accession', 'Name', 'PubChem_CID'])
    df = df.append(temp_df)
    
df.index = range(0, len(df))        
    
#cid 존재하는 hmdb
df_hmdb_cid = df[~df.PubChem_CID.isnull()]

# using UniChem REST API, convert pubchem ID to KEGG ID
df_hmdb = pd.DataFrame()
url_base = 'https://www.ebi.ac.uk/unichem/rest/src_compound_id/'
cnt = 0
print "pubchem ids of hmdb are converted to KEGG ID start : " + time.strftime('%a %H:%M:%S')
for index, row in df_hmdb_cid.iterrows():
    accession = row['Accession']
    name = row['Name']
    hmdb_pubchem_id = row['PubChem_CID']
        
    url = url_base + hmdb_pubchem_id + '/22/6'
    
    MAX_ATTEMPTS = 8
    for attempt in range(MAX_ATTEMPTS):
        try:
            f = urlopen(url)
        except urllib2.HTTPError, e:
            if e.code == 404:
                cnt = cnt + 1
                break
            else:
                print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                raise
        except urllib2.URLError, e:
            if e.args[0][0] == 110: # connection time out
                sleep_secs = attempt ** 2
                print e
                print url
                print '  cnt = %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                print '    Retrying in %d seconds...' % sleep_secs
                time.sleep(sleep_secs)
                continue
            else:
                print e
                print '   cnt : %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                raise
        else: # error가 일어나지 않을 때.
            raw_data = f.readlines()
            f.close()
            cnt = cnt + 1
            if raw_data[0] == '[]':
                break
            else:
                splited = raw_data[0].rstrip(']').lstrip('[').split(',')
                for i in splited:
                    kegg_id = i.split(':')[1].lstrip("\"").rstrip("\"}")
                    if kegg_id != 'null':
                        temp_df = pd.DataFrame([[accession, name, hmdb_pubchem_id, kegg_id]], columns = ['Accession', 'Name', 'PubChem_CID', 'KEGG_ID'])
                        df_hmdb = df_hmdb.append(temp_df)
            
            break 
            
    else: #connection을 8번 시도했는데 안될 경우.
        print 'we failed to reconnect 8 times in %s' % url 
    if cnt % 1000 == 0 :
        print "%d complete "%cnt + time.strftime('%a %H:%M:%S')
print "end : " + time.strftime('%a %H:%M:%S')
    
df_hmdb.index = range(len(df_hmdb))

        
        

preprocess the HMDB files
pubchem ids of hmdb are converted to KEGG ID start : Thu 17:21:36
1000 complete Thu 17:39:00
2000 complete Thu 17:56:23
3000 complete Thu 18:14:18
4000 complete Thu 18:32:03
5000 complete Thu 18:49:49
<urlopen error [Errno 110] Connection timed out>
https://www.ebi.ac.uk/unichem/rest/src_compound_id/53479672/22/6
  cnt = 5079, error time : Thu 18:53:20
    Retrying in 0 seconds...
6000 complete Thu 19:09:42
7000 complete Thu 19:27:34
8000 complete Thu 19:45:41
9000 complete Thu 20:03:54
10000 complete Thu 20:22:14
11000 complete Thu 20:40:45
12000 complete Thu 20:59:04
13000 complete Thu 21:17:13
14000 complete Thu 21:35:17
15000 complete Thu 21:53:18
16000 complete Thu 22:11:20
end : Thu 22:28:02


In [57]:
print len(df_hmdb_cid)
print len(df_hmdb)
print cnt

16923
3147
16923


In [58]:
df_hmdb.to_csv("/home/share/aschoi/data/COCONUT_Herbal/4_HMDB/HMDB_KEGG_new.tsv", sep="\t", index=False)

In [60]:
# Make the final form of herbfiles. 'herb_path' is the location which herb file is in. (tsv format)
# 'stitch_cid_path' is the path of the STITCH-PubChem CID id mapping file.
# raw herb file comes from COCONUT database. columns : (herbID), (herbName), compID, compName, pubchemID, casID, chemblID, stitchID
herb_path = "/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/COCONUT_herb_compound_ID(remove_duplicate)_tab.txt"
stitch_cid_path = "/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/STITCH_CID.tsv"
def preprocess_raw_herb(herb_path, stitch_cid_path):
    print "preprocess the raw herb file"
    
    df = pd.read_table(herb_path)
    df = df.fillna('null')
    
    ############ pubchemID ############
    df_herb = df[df.pubchemID != 'null'][['compID', 'compName', 'pubchemID']]
    
    ########### stitch #################
    df_stitch = df[(df.pubchemID =='null') & (df.stitchID != 'null')]
    
    # read the pair of STITCH ID - PubChem CID
    df_stitch_cid= pd.read_table(stitch_cid_path, index_col = False)
    
    # find the PubChem CID of STITCH id from STITCH ID - PubChem CID pair
    for index, row in df_stitch.iterrows():
        compID = row['compID']
        compName = row['compName']
        stitchID = row['stitchID']
        
        found = df_stitch_cid[df_stitch_cid.stereo_chemical == stitchID]
        pubchemID = found['source_id']
        #temp_df = pd.DataFrame([[compID, compName, pubchemID]])
    
    # read the pair of STITCH ID - PubChem CID
    df_stitch_cid= pd.read_table(stitch_cid_path, index_col = False)

    # find the PubChem CID of STITCH id from STITCH ID - PubChem CID pair
    for index, row in df_stitch.iterrows():
        compID = row['compID']
        compName = row['compName']
        stitchID = row['stitchID']
            
        found = df_stitch_cid[df_stitch_cid.stereo_chemical == stitchID]
        pubchemID = found['source_id']
        temp_df = pd.DataFrame({'compID' : compID, 'compName' : compName, 'pubchemID' : pubchemID})
        df_herb = df_herb.append(temp_df)
    
    ########### cas ID ##############
    df2_flag = df['compID'].isin(df_herb['compID'].drop_duplicates())
    df2 = df[~df2_flag]
    df_cas = df2[(df2.pubchemID == 'null') & (df2.casID != 'null')]
    
    temp = df_cas[df_cas['casID'].str.contains('/')]
    for index, row in temp.iterrows():
        cas_splited = row['casID'].split('/')
        if len(cas_splited[0]) < 2:
            cas = cas_splited[2] + '-0' + cas_splited[0] + '-' + cas_splited[1] 
        else:
            cas = cas_splited[2] + '-' + cas_splited[0] + '-' + cas_splited[1] 
        df_cas.set_value(index,'casID', cas)
    
    # using chemical translation service REST webservices, convert casID to KEGG ID    
    cnt = 0
    url_base = 'http://cts.fiehnlab.ucdavis.edu/service/convert/CAS/PubChem%20CID/'    
    print "cas ids of herb compound are converted into pubchem ids start : " + time.strftime('%a %H:%M:%S')
    for index, row in df_cas.iterrows():
        compID = row['compID']
        compName = row['compName']
        casID = row['casID']
        
        url = url_base + casID
                   
        MAX_ATTEMPTS = 8
        for attempt in range(MAX_ATTEMPTS):
            try:
                f = urlopen(url)
            except urllib2.URLError, e:
                if e.args[0][0] == 110: # connection time out
                    sleep_secs = attempt ** 2
                    print e
                    print url
                    print '  cnt = %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    print '    Retrying in %d seconds...' % sleep_secs
                    time.sleep(sleep_secs)
                    continue
                else:
                    print e
                    print '   cnt : %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    raise
            else: # error가 일어나지 않을 때.
                raw_data = json.load(f)
                res = raw_data[0]['result']
                if res == []: # 못찾음
                    cnt = cnt + 1
                else:
                    cnt = cnt + 1
                    pubchemID = res[0].encode("ascii", "replace")
                    temp_df = pd.DataFrame([[compID, compName, pubchemID]], columns = ['compID','compName','pubchemID'])
                    df_herb = df_herb.append(temp_df)
                break
        else: #connection을 8번 시도했는데 안될 경우.
            print 'we failed to reconnect 8 times in %s' % url 
    print "end : " + time.strftime('%a %H:%M:%S')
    
    df_herb.index = range(len(df_herb))
    return df_herb

In [4]:
#open files
df_raw_herb_hmdb_pair = pd.read_csv('/home/share/aschoi/data/COCONUT_Herbal/5_Similarity/7_sorted_score_vector_except_1(descending)_top_10_percent_no_score.csv', header=None)
#df_hmdb = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/4_HMDB/HMDB_KEGG.tsv')
#df_herb = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/2-5_herb_pubchem.tsv')
df_kegg_rpair = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/1_KEGG_rpair/KEGG_Rpair.txt')
df_orthology = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/5_predicted_orthology.tsv')

In [5]:
# extract elements of top percentage.
def count_percentage(percentage):
    print"  Count the number of top %.2f %% similar compounds.." % percentage
    WHOLE_LENGTH = 646647053
    prepared_idx = 0
    percentage_idx = int(WHOLE_LENGTH * percentage / 100)
    
    if percentage <= 0.1 and percentage > 0 :
        df_top_herb_hmdb_pair = df_raw_herb_hmdb_pair[:percentage_idx]
        connect = 0
        print "  -> There are %d similar compounds." % (prepared_idx + percentage_idx)
        #print "len(df_top_herb_hmdb_pair : %d)" %len(df_top_herb_hmdb_pair)
    elif percentage == 0 or percentage > 10 :
        #if percentage > 10 :
            #print("Input percentage is too big. Count just Top 10%.")
        #else percentage == 0 :
            #print("Input percentage is not valid.")
        df_top_herb_hmdb_pair = pd.DataFrame()
        connect = 1
    else :
        prepared_idx = int(WHOLE_LENGTH * 0.1 / 100)
        df_top_herb_hmdb_pair = df_raw_herb_hmdb_pair[prepared_idx+1:percentage_idx]
        connect = 2
        print "There are %d similar compounds." % (prepared_idx + percentage_idx)
    #print"len(df_top_herb_hmdb_pair) : %d" % len(df_top_herb_hmdb_pair)
    return connect, df_top_herb_hmdb_pair

In [6]:
#top herb id, top hmdb id
def extract_top_id(df_top_herb_hmdb_pair):
    s_top_herb = df_top_herb_hmdb_pair['compID']
    s_top_hmdb = df_top_herb_hmdb_pair['Accession']
    print("    1. Extract the ids of top herbal and top HMDB compounds.")
    #print("           len(s_top_herb, hmdb) : %d %d")%(len(s_top_herb), len(s_top_hmdb))
    return s_top_herb, s_top_hmdb

In [7]:
def find_keggid_of_hmdb(df_hmdb, s_top_hmdb):
    # The flag is used to find the KEGG ID of top_similar_hmdb.
    flag_find_keggid_of_top_hmdb = df_hmdb['Accession'].isin(s_top_hmdb)

    # This dataframe represents the pair of top hmdb id and its KEGG ID.
    # This information is used when find the substrate of KEGG rpair which correspond to the top_hmdb.
    df_top_hmdb_keggid_pair = df_hmdb[flag_find_keggid_of_top_hmdb][['Accession', 'KEGG_ID']]
    print("    2. Find the KEGG id of whole top HMDB compounds.")
    #print("        len(df_top_hmdb_keggid_pair) : %d") % len(df_top_hmdb_keggid_pair)
    return df_top_hmdb_keggid_pair

In [8]:
glob_top_herb_pubchemid_pair = 0;
def find_pubchemid_of_herb(df_herb, s_top_herb):
    # The flag is used to find the PubChem ID of top_herb.
    flag_find_pubchemid_of_top_herb = df_herb['compID'].isin(s_top_herb)

    # This dataframe represents the pair of top herb id and its PubChem ID.
    # Using this information, we can predict the orphan herbs' ontology.
    # One pubchem id of herb becomes the input substrate of KEGG E-zyme2 tool.
    df_top_herb_pubchemid_pair = df_herb[flag_find_pubchemid_of_top_herb][['compID', 'pubchemID']]
    print("    3. Find the PubChem id of top herbal compounds.")
    #print("        len(df_top_herb_pubchemid_pair) : %d") % len(df_top_herb_pubchemid_pair)
    glob_top_herb_pubchemid_pair = df_top_herb_pubchemid_pair
    return df_top_herb_pubchemid_pair

In [9]:
# Find the rpairs whose substrate is same to the top_hmdb.
# The pair is represented by only KEGG ID. (NOT hmdb_id.)
def find_kegg_rpair_of_hmdb_sub(df_kegg_rpair, df_top_hmdb_keggid_pair):
    flag_find_rpair_of_top_hmdb_sub = df_kegg_rpair['Compound1'].isin(df_top_hmdb_keggid_pair['KEGG_ID']) 

    # This dataframe consist of KEGG ID.
    df_rpair_of_top_hmdb_sub_k = df_kegg_rpair[flag_find_rpair_of_top_hmdb_sub]
    print("    4. Find the KEGG rpair whose substrate is same to the top hmdb compounds.")
    #print("        len(df_rpair_of_top_hmdb_sub_k) : %d") % len(df_rpair_of_top_hmdb_sub_k)
    #print sum(flag_find_rpair_of_top_hmdb_sub)
    #print len(df_rpair_of_top_hmdb_sub_k)
    #print len(df_rpair_of_top_hmdb_sub_k['Compound1'].drop_duplicates())
    return df_rpair_of_top_hmdb_sub_k

In [10]:
# Find the part of top_hmdb. 
# It have KEGG ID which corresponds to the substrate of KEGG rpair.
# Why need this process? 
# Because we don't have the pair of the kegg_id version substrate(although it corresponds to the top_hmdb!) and top_hmdb id.

def find_keggid_of_top_hmdb2(df_top_hmdb_keggid_pair, df_rpair_of_top_hmdb_sub_k):

    flag_find_hmdbid_of_top_hmdb_sub = df_top_hmdb_keggid_pair['KEGG_ID'].isin(df_rpair_of_top_hmdb_sub_k['Compound1'])
    df_top_hmdb_of_sub = df_top_hmdb_keggid_pair[flag_find_hmdbid_of_top_hmdb_sub]
    print("    5. Find some HMDB compounds which corresponds to the substrate of KEGG rpair.")
    #print("        len(df_top_hmdb_of_sub) : %d") % len(df_top_hmdb_of_sub)
    #print sum(flag_find_hmdbid_of_top_hmdb_sub)
    #print len(df_top_hmdb_of_sub)
    #print len(df_top_hmdb_of_sub['Accession'].drop_duplicates())
    #print len(df_top_hmdb_of_sub['KEGG_ID'].drop_duplicates())
    return df_top_hmdb_of_sub

In [11]:
# Find the pair of top_hmdb and top_herb.
# The top_hmdb is the substrate of the KEGG rpair.
# The top_herb is the similar herbal compounds of the top_hmdb.
def find_candidate_substrate(df_top_herb_hmdb_pair, df_top_hmdb_of_sub, df_top_herb_pubchemid_pair):
    flag_find_sim_herb_of_hmdb_sub = df_top_herb_hmdb_pair['Accession'].isin(df_top_hmdb_of_sub['Accession'])
    df_sim_herb_of_hmdb_sub = df_top_herb_hmdb_pair[flag_find_sim_herb_of_hmdb_sub]
    
    # Find the pubchem id of the herbs which is similar to the hmdb compounds. These hmdb compounds are the substrate of rpair. 
    flag_find_pubchemid_of_sim_herb =df_top_herb_pubchemid_pair['compID'].isin(df_sim_herb_of_hmdb_sub['compID'])
    df_sim_herb_pubchem_id_pair = df_top_herb_pubchemid_pair[flag_find_pubchemid_of_sim_herb]

    df_candidate_sub_herb = pd.merge(df_top_hmdb_of_sub, df_sim_herb_of_hmdb_sub, how = 'inner', on = ['Accession', 'Accession'])
    df_candidate_sub_herb_pubchem = pd.merge(df_candidate_sub_herb, df_sim_herb_pubchem_id_pair, how='inner', on=['compID', 'compID'])
    
    print("    6. Find the candidate herbal compounds which is similar to HMDB's substrates.")
    #print("        len(df_candidate_sub_herb_pubchem) : %d") % len(df_candidate_sub_herb_pubchem)
    return df_candidate_sub_herb_pubchem

In [12]:
# Make the pair of herb(used to be substrate) and product.
# If the pair will exist in the prepared orthology table, just show the predicted_result.
# Unless the pair will not exist in the prepared table, we have to crawl the predicted reusult from KEGG E-zyme2 webpage.
def query_list(df_candidate_sub_herb_pubchem, df_rpair_of_top_hmdb_sub_k):
    #print "       inner query_list : len(df_candidate_sub_herb_pubchem, df_rpair_of_top_hmdb_sub_k) %d, %d" % (len(df_candidate_sub_herb_pubchem), len(df_rpair_of_top_hmdb_sub_k))
    s_candidate_keggid = df_candidate_sub_herb_pubchem['KEGG_ID'].drop_duplicates()
    #print "       len(s_candidate_keggid) : %d" % len(s_candidate_keggid)
    df_query = pd.DataFrame()
    for i in s_candidate_keggid:
        df_candidate_sub_part = df_candidate_sub_herb_pubchem[df_candidate_sub_herb_pubchem.KEGG_ID == i]
        df_candidate_pro_part = df_rpair_of_top_hmdb_sub_k[df_rpair_of_top_hmdb_sub_k.Compound1 == i]
        s_candidate_sub_part = df_candidate_sub_part['pubchemID']
        s_candidate_pro_part = df_candidate_pro_part['Compound2']
        temp_df = pd.DataFrame(list(product(s_candidate_sub_part, s_candidate_pro_part)), columns = ['substrate', 'product'])
        df_query = df_query.append(temp_df)
    df_query = df_query.drop_duplicates()
    df_query.index = range(0, len(df_query))
    print("    7. Make the query lists of the candidate rpair which will be predicted.")
    #print("        len(df_query) : %d") % len(df_query)
    return df_query

In [13]:
def match_query_result(df_query, df_orthology):
    #print("start match query result : " + time.strftime('%a %H:%M:%S'))
    predict_result = pd.DataFrame()
    for index, row in df_query.iterrows():
        query = row.tolist()
        #result = df_orthology.query('@query[0] == substrate and @query[1] == product')
        result = df_orthology[(df_orthology.substrate == query[0]) & (df_orthology['product'].str.match(query[1]))]
        if result.empty:
            continue
        else:
            predict_result = predict_result.append(result)
            #pdb.set_trace()
    predict_result.index = range(0, len(predict_result))    
    #print("end match query result" + time.strftime('%a %H:%M:%S'))
    print("    8. Find the predicted orthology of the query rpair.")
    return predict_result

In [14]:
#def make_query_list(top_herb, top_hmdb):
def make_query_list(df_top_herb_hmdb_pair):
    
    
    df_top_herb_hmdb_pair = df_top_herb_hmdb_pair.rename(columns={0:'compID', 1:'Accession'})
    s_top_herb, s_top_hmdb = extract_top_id(df_top_herb_hmdb_pair)
    
    # Find the KEGG ID of top_hmdb.
    df_top_hmdb_keggid_pair = find_keggid_of_hmdb(df_hmdb, s_top_hmdb)
    
    # Find the PubChem ID of top_herb.
    df_top_herb_pubchemid_pair = find_pubchemid_of_herb(df_herb, s_top_herb)
    
    # Find the rpairs whose substrate is same to the top_hmdb.
    df_rpair_of_top_hmdb_sub_k = find_kegg_rpair_of_hmdb_sub(df_kegg_rpair, df_top_hmdb_keggid_pair)
    
    # Find the part of top_hmdb. 
    df_top_hmdb_of_sub = find_keggid_of_top_hmdb2(df_top_hmdb_keggid_pair, df_rpair_of_top_hmdb_sub_k)
    
    # Find the pair of top_hmdb and top_herb.
    df_candidate_sub_herb_pubchem = find_candidate_substrate(df_top_herb_hmdb_pair, df_top_hmdb_of_sub, df_top_herb_pubchemid_pair)
    #print "       before query_list : len(df_candidate_sub_herb_pubchem, df_rpair_of_top_hmdb_sub_k) %d, %d" % (len(df_candidate_sub_herb_pubchem), len(df_rpair_of_top_hmdb_sub_k))
    # Make the pair of herb(used to be substrate) and product.
    df_query = query_list(df_candidate_sub_herb_pubchem, df_rpair_of_top_hmdb_sub_k)
    
    return df_query

In [15]:
def show_predicted_result(predicted_result, df_herb):
    herb_predicted_result = pd.merge(left = df_herb, right = predicted_result, left_on = 'pubchemID', right_on = 'substrate')
    herb_predicted_result = herb_predicted_result.rename(columns={'compID':'COCONUT_herb_compound_id',
                                                                  'compName':'COCONUT_herb_compound_name',
                                                                  'pubchemID' : 'COCONUT_herb_compound_PubChem_id' })
    print herb_predicted_result
    return herb_predicted_result

In [16]:
def web_ezyme(df_query):
    print "    9. The number of crawling pairs : %d " % len(df_query)
    predicted_result2 = pd.DataFrame()
    cnt = 0
    url_base = 'http://rest.genome.jp/ezyme/'
    
    for index, row in df_query[cnt:].iterrows():
        sub = int(row['substrate'])
        pro = row['product']
        url = url_base + 'CID'+str(sub) +'/' + pro
        
        MAX_ATTEMPTS = 8
        for attempt in range(MAX_ATTEMPTS):
            try:
                f = urlopen(url)
            except urllib2.HTTPError, e:
                if e.code == 404:
                    print 'HTTPError 404 not found %d : ' % cnt + time.strftime('%a %H:%M:%S')
                    print url
                    cnt = cnt + 1
                    break
                else:
                    print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                    raise
            except urllib2.URLError, e:
                if e.args[0][0] == 110: # connection time out
                    sleep_secs = attempt ** 2
                    print e
                    print '  cnt = %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    print urll
                    print '    Retrying in %d seconds...' % sleep_secs
                    time.sleep(sleep_secs)
                    continue
                else:
                    print e
                    print '   cnt : %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    raise
            else:          
                all_data = f.readlines()
                data = all_data[1].split('\t')
                predicted_orthology = data[1]
                predicted_score = data[0]        
                temp_df = pd.DataFrame([[sub, pro, predicted_orthology, predicted_score]], columns = ['substrate', 'product', 'predicted_orthology', 'predicted_score'])
                predicted_result2 = predicted_result2.append(temp_df)
                cnt = cnt + 1
                break
        else:
            print 'we failed to reconnect 8 times in %s' % url 
        
        #if cnt % 1000 == 0 :
        #    print '%d : '% cnt + time.strftime('%a %H:%M:%S')
    #print time.strftime('%a %H:%M:%S')
    predicted_result2.index = range(0, len(predicted_result2))
    return predicted_result2
    

In [22]:
hmdb_path = "/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/4_HMDB/hmdb_metabolites/"
df_hmdb =  preprocess_raw_hmdb(hmdb_path)

preprocess the HMDB files
pubchem ids of hmdb are converted to KEGG ID start : Thu 10:07:23
we failed to reconnect 8 times in https://www.ebi.ac.uk/unichem/rest/src_compound_id/92105/22/6
we failed to reconnect 8 times in https://www.ebi.ac.uk/unichem/rest/src_compound_id/11266/22/6
we failed to reconnect 8 times in https://www.ebi.ac.uk/unichem/rest/src_compound_id/53477676/22/6
we failed to reconnect 8 times in https://www.ebi.ac.uk/unichem/rest/src_compound_id/6451121/22/6
we failed to reconnect 8 times in https://www.ebi.ac.uk/unichem/rest/src_compound_id/1125/22/6
we failed to reconnect 8 times in https://www.ebi.ac.uk/unichem/rest/src_compound_id/12306765/22/6
we failed to reconnect 8 times in https://www.ebi.ac.uk/unichem/rest/src_compound_id/440567/22/6
we failed to reconnect 8 times in https://www.ebi.ac.uk/unichem/rest/src_compound_id/24758425/22/6
we failed to reconnect 8 times in https://www.ebi.ac.uk/unichem/rest/src_compound_id/657311/22/6
we failed to reconnect 8 times i

In [61]:
herb_path = "/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/COCONUT_herb_compound_ID(remove_duplicate)_tab.txt"
stitch_cid_path = "/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/STITCH_CID.tsv"
df_herb = preprocess_raw_herb(herb_path, stitch_cid_path)

preprocess the raw herb file
cas ids of herb compound are converted into pubchem ids start : Thu 23:43:04
end : Fri 00:05:09


In [62]:
print len(df_herb)
df_herb.head()

11371


,compID,compName,pubchemID
0,10237,Phenol,20488062
1,52815,"(2R,3R)-2-(3,4-Dihydroxyphenyl)-3,7-dihydroxy-...",10343835
2,62077,"(2E)-3-{2-Hydroxy-4-[(2R,3R)-3-(hydroxymethyl)...",6438693
3,62076,"(13alpha,14beta,17alpha,20S,21R,23R,24S)-21-Hy...",44575793
4,67348,"(5xi,9xi,13xi,14xi,17xi,20xi)-21-Acetoxy-21,23...",5321231


In [ ]:
###### Main ###########

# aceeptuser input
percentage = input('Enter the percentage number (0,10] : ')
connect, df_top_herb_hmdb_pair = count_percentage(percentage)

if connect == 1: #Input is not valid.
    print("Input is not valid. Stop the program.")

else:
    df_query = make_query_list(df_top_herb_hmdb_pair)
    
    if connect == 2: # Web crawling
        print("Need to connect the web. time consuming.. : " + time.strftime('%a %H:%M:%S'))
        predicted_result = web_ezyme(df_query)
        print("End of web crawling : " + time.strftime('%a %H:%M:%S'))
        predicted_result = predicted_result.append(df_orthology) # prepared result (percentage under 0.1%)
        print("There are %d predicted orthology results.") % len(predicted_result)
    
    else: # Matching
        if percentage == 0.1:
            predicted_result = df_orthology
        else:
            predicted_result = match_query_result(df_query, df_orthology)
        print("There are %d predicted orthology results.") % len(predicted_result)
        predictied_result.to_csv("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/e_zyme_tool_output.tsv", sep='\t',index=False)
        #a = show_predicted_result(predicted_result, df_herb)

In [ ]:
herb_path = "/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/COCONUT_herb_compound_ID(remove_duplicate)_tab.txt"
stitch_cid_path = "/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/STITCH_CID.tsv"

df = pd.read_table(herb_path)
df_stitch = df[(df.pubchemID =='null') & (df.stitchID != 'null')]

# read the pair of STITCH ID - PubChem CID
df_stitch_cid= pd.read_table(stitch_cid_path, index_col = False)

df_herb = pd.DataFrame()

# find the PubChem CID of STITCH id from STITCH ID - PubChem CID pair
for index, row in df_stitch.iterrows():
    compID = row['compID']
    compName = row['compName']
    stitchID = row['stitchID']
        
    found = df_stitch_cid[df_stitch_cid.stereo_chemical == stitchID]
    pubchemID = found['source_id']
    temp_df = pd.DataFrame({'compID' : compID, 'compName' : compName, 'pubchemID' : pubchemID})
    df_herb = df_herb.append(temp_df)

In [ ]:
#cas ID
df2_flag = df['compID'].isin(df_herb['compID'].drop_duplicates())
df2 = df[~df2_flag]
df_cas = df2[(df2.pubchemID == 'null') & (df2.casID != 'null')]

temp = df_cas[df_cas['casID'].str.contains('/')]
for index, row in temp.iterrows():
    cas_splited = row['casID'].split('/')
    if len(cas_splited[0]) < 2:
        cas = cas_splited[2] + '-0' + cas_splited[0] + '-' + cas_splited[1] 
    else:
        cas = cas_splited[2] + '-' + cas_splited[0] + '-' + cas_splited[1] 
    df_cas.set_value(index,'casID', cas)
    
# using chemical translation service REST webservices, convert casID to KEGG ID    
cnt = 0
url_base = 'http://cts.fiehnlab.ucdavis.edu/service/convert/CAS/PubChem%20CID/'    
print time.strftime('%a %H:%M:%S')
for index, row in df_cas.iterrows():
        compID = row['compID']
        compName = row['compName']
        casID = row['casID']
        
        url = url_base + casID
                   
        MAX_ATTEMPTS = 8
        for attempt in range(MAX_ATTEMPTS):
            try:
                f = urlopen(url)
            except urllib2.URLError, e:
                if e.args[0][0] == 110: # connection time out
                    sleep_secs = attempt ** 2
                    print e
                    print url
                    print '  cnt = %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    print '    Retrying in %d seconds...' % sleep_secs
                    time.sleep(sleep_secs)
                    continue
                else:
                    print e
                    print '   cnt : %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    raise
            else: # error가 일어나지 않을 때.
                raw_data = json.load(f)
                res = raw_data[0]['result']
                if res == []: # 못찾음
                    cnt = cnt + 1
                else:
                    cnt = cnt + 1
                    pubchemID = res[0].encode("ascii", "replace")
                    temp_df = pd.DataFrame([[compID, compName, pubchemID]], columns = ['compID','compName','pubchemID'])
                    df_hmdb.append(temp_df)
                break
        else: #connection을 8번 시도했는데 안될 경우.
            print 'we failed to reconnect 8 times in %s' % url 
print time.strftime('%a %H:%M:%S')